In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import tree
from sklearn.feature_selection import f_classif
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Učitavanje dataset-a
df = pd.read_csv('diabetes_data.csv')
df = pd.DataFrame(df)

In [ ]:
plt.hist(df['Age'], bins=10, edgecolor='black')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Histogram of Age')
plt.show()

In [ ]:
gender_counts = df['Gender'].value_counts()
plt.bar(gender_counts.index, gender_counts.values)
plt.xlabel('Gender')
plt.ylabel('Count')
plt.title('Bar Chart of Gender')
plt.show()

In [ ]:
# Predprocesiranje podataka
label_encoder = LabelEncoder()
binary_columns = ['Gender','Polyuria', 'Polydipsia', 'sudden weight loss', 'weakness', 'Polyphagia', 'Genital thrush',
                  'visual blurring', 'Itching', 'Irritability', 'delayed healing', 'partial paresis',
                  'muscle stiffness', 'Alopecia', 'Obesity', 'class']

for column in binary_columns:
    df[column] = label_encoder.fit_transform(df[column])

In [ ]:
df

In [ ]:
X = df.drop(['class'], axis=1)
y = df['class']

# Podjela podataka na skupove za treniranje i testiranje
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=17)

In [ ]:
# Primjena ANOVA testiranja hipoteza i odabir k atributa
def select_features(X_train, y_train, k):
    f_scores, p_values = f_classif(X_train, y_train)
    selected_indices = np.argsort(f_scores)[-k:]
    return selected_indices

In [ ]:
# Konstrukcija stabla odlučivanja na osnovu odabranih atributa
def build_decision_tree(X_train, y_train):
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    return clf

In [ ]:
# Odabir atributa pomoću ANOVA testiranja hipoteza
selected_indices = select_features(X_train, y_train, 10)

In [ ]:
# Odabir samo odabranih atributa za treniranje i testiranje
X_train_selected = X_train.iloc[:, selected_indices]
X_test_selected = X_test.iloc[:, selected_indices]


In [ ]:
X_test_selected

In [ ]:
# Konstrukcija stabla odlučivanja na osnovu odabranih atributa
clf = build_decision_tree(X_train_selected, y_train)

In [ ]:
# Ispis stabla odlučivanja
fig = plt.figure(figsize=(35, 35))
_ = tree.plot_tree(clf, feature_names=X.columns, class_names=['False', 'True'], filled=True)
plt.show()

In [ ]:
# Predikcija testnog skupa
y_pred = clf.predict(X_test_selected)
y_pred

In [ ]:

# Evaluacija Klasifikatora


In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.show()

In [ ]:
# Tačnost predikcije
accuracy = accuracy_score(y_test, y_pred)
print("Tačnost predikcije: ", accuracy)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Izračunajte konfuzijsku matricu
cm = confusion_matrix(y_test, y_pred)

# Kreirajte heatmapu konfuzijske matrice
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")

# Dodajte oznake osi i naslov
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")

# Prikazati grafik
plt.show()
'''
Gornji lijevi element (53) predstavlja broj uzoraka koji pripadaju negativnoj klasi (stvarno negativni) i koji su ispravno predviđeni kao negativni.
Gornji desni element (2) predstavlja broj uzoraka koji pripadaju negativnoj klasi (stvarno negativni), ali su pogrešno predviđeni kao pozitivni.
Donji lijevi element (6) predstavlja broj uzoraka koji pripadaju pozitivnoj klasi (stvarno pozitivni), ali su pogrešno predviđeni kao negativni.
Donji desni element (69) predstavlja broj uzoraka koji pripadaju pozitivnoj klasi (stvarno pozitivni) i koji su ispravno predviđeni kao pozitivni.
'''

In [ ]:
report = classification_report(y_test, y_pred)
print("Izvještaj klasifikacije:\n", report)

In [ ]:
# Izvršavanje klasifikacije nad jednim podatkom 0:no 1:yes
new_data = {
    'Alopecia': [0], 'weakness': [1], 'visual blurring': [1], 'Irritability': [1], 'Polyphagia': [0], 'sudden weight loss': [1], 'partial paresis': [1], 'Gender': [0], 'Polydipsia': [0], 'Polyuria': [1], 'class': [1]
}

new_data_df = pd.DataFrame(new_data)

# Izdvajanje karakteristika za predikciju
X_new = new_data_df.drop(['class'], axis=1)
# Izvršavanje predikcije na novim podacima
y_new_pred = clf.predict(X_new)

# Ispis predikcije
print("Da li ćosoba potencijalno ima dijabetes: ", y_new_pred)